In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
prashantarorat_facial_key_point_data_path = kagglehub.dataset_download('prashantarorat/facial-key-point-data')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/facial-key-point-data/all_data.json
/kaggle/input/facial-key-point-data/images/01301.png
/kaggle/input/facial-key-point-data/images/02578.png
/kaggle/input/facial-key-point-data/images/00929.png
/kaggle/input/facial-key-point-data/images/04915.png
/kaggle/input/facial-key-point-data/images/02630.png
/kaggle/input/facial-key-point-data/images/03862.png
/kaggle/input/facial-key-point-data/images/01883.png
/kaggle/input/facial-key-point-data/images/00704.png
/kaggle/input/facial-key-point-data/images/03163.png
/kaggle/input/facial-key-point-data/images/01124.png
/kaggle/input/facial-key-point-data/images/04159.png
/kaggle/input/facial-key-point-data/images/01570.png
/kaggle/input/facial-key-point-data/images/03092.png
/kaggle/input/facial-key-point-data/images/03663.png
/kaggle/input/facial-key-point-data/images/04084.png
/kaggle/input/facial-key-point-data/images/04216.png
/kaggle/input/facial-key-point-data/images/02640.png
/kaggle/input/facial-key-point-data/images/02422.

In [ ]:
import cv2
import os
from PIL import Image

# Define the paths
input_folder = '/kaggle/input/facial-key-point-data/images'
new_size = (256, 256)  # Desired size

# Load the pre-trained Haar Cascade model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    landmarks = []
    for (x, y, w, h) in faces:
        # Extract the center of the bounding box as a simple landmark
        center_x = x + w // 2
        center_y = y + h // 2
        landmarks.append((center_x, center_y))
    return landmarks

# Define the maximum number of landmarks expected
MAX_LANDMARKS = 68  # Change this based on your requirements
landmark_dim = 2    # Each landmark as (x, y)

# Lists to store processed images and landmarks
processed_images = []
all_landmarks = []

# Iterate over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path)
        img = img.resize(new_size, Image.LANCZOS)  # Resize the image using LANCZOS filter

        img_np = np.array(img)  # Convert image to numpy array

        # Detect landmarks
        landmarks = get_landmarks(img_np)

        # Pad landmarks if necessary
        if len(landmarks) < MAX_LANDMARKS:
            landmarks.extend([(0, 0)] * (MAX_LANDMARKS - len(landmarks)))
        else:
            landmarks = landmarks[:MAX_LANDMARKS]

        all_landmarks.append(landmarks)

        # Append resized image to list
        processed_images.append(img_np)

# Convert lists to numpy arrays
processed_images_np = np.array(processed_images) / 255.0  # Normalize images
all_landmarks_np = np.array(all_landmarks)

# Save the processed data
np.save("processed_images.npy", processed_images_np)  # Save images
np.save("facial_landmarks.npy", all_landmarks_np)  # Save landmarks

# Output sizes of the saved arrays
print(f"Processed images shape: {processed_images_np.shape}")
print(f"Facial landmarks shape: {all_landmarks_np.shape}")


Processed images shape: (5000, 256, 256, 3)
Facial landmarks shape: (5000, 68, 2)


In [ ]:
!pip install keras-tuner  --upgrade

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import keras_tuner as kt
from sklearn.model_selection import train_test_split

# Assuming your dataset is available in the Kaggle environment
# If your data is saved as npy files, you can load them like this
processed_images = np.load("processed_images.npy")  # Update the path if needed
facial_landmarks = np.load("facial_landmarks.npy")  # Update the path if needed

In [ ]:
# Reshape facial landmarks to (5000, 136) to match output of Dense layer
facial_landmarks = facial_landmarks.reshape((facial_landmarks.shape[0], 68 * 2))

# Split the dataset into training and validation
x_train, x_val, y_train, y_val = train_test_split(processed_images, facial_landmarks, test_size=0.2, random_state=42)


In [ ]:
# Function to build the model with Keras Tuner
def build_model(hp):
    model = models.Sequential()

    # Add Convolutional layers
    for i in range(hp.Int('conv_blocks', 1, 3, default=2)):
        model.add(layers.Conv2D(
            filters=hp.Int(f'filters_{i}', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice(f'kernel_size_{i}', values=[3, 5]),
            activation='relu',
            input_shape=(256, 256, 3) if i == 0 else None
        ))
        model.add(layers.MaxPooling2D(pool_size=2))

    # Flatten before Dense layers
    model.add(layers.Flatten())

    # Add fully connected layers
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))

    # Output layer for 68 landmarks (136 values)
    model.add(layers.Dense(136))


    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='mean_squared_error',
                  metrics=['mae'])

    return model

In [ ]:
# Set up the Keras Tuner for hyperparameter optimization
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',  # We aim to minimize the Mean Absolute Error
    max_trials=2,         # Number of different models to try
    executions_per_trial=1,  # Number of times to train each model (1 for speed)
    directory='/kaggle/working/nas_landmark_detection',  # Save the search results here
    project_name='cnn_nas'  # Name of the project
)

# Summary of search space
tuner.search_space_summary()

# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Get the best model after the search
best_model = tuner.get_best_models(num_models=1)[0]

# Display a summary of the best model
best_model.summary()


Trial 2 Complete [00h 02m 04s]
val_mae: 0.6321548819541931

Best val_mae So Far: 0.6321548819541931
Total elapsed time: 00h 07m 32s


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 252, 252, 64)   │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 492032)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │   125,960,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 136)            │        34,952 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,074,120 (480.93 MB)

 Trainable params: 126,074,120 (480.93 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Optionally, evaluate the best model on the validation set
val_loss, val_mae = best_model.evaluate(x_val, y_val)
print(f"Validation Loss: {val_loss}, Validation MAE: {val_mae}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 25.9029 - mae: 0.6330
Validation Loss: 26.350666046142578, Validation MAE: 0.6321548819541931
